# 플랫폼 업로드를 쉽게하기 위한 로컬 개발 코드
- T3Q.ai(T3Q.cep + T3Q.dl): 빅데이터/인공지능 통합 플랫폼
- 플랫폼 업로드를 쉽게하기 위하여 로컬에서 2개의 파일로 나누어 코드를 개발한다.
- 파일 1_1: 전처리와 학습모델을 개발하는 코드, 전처리 객체나 성능이 검증된 학습모델 객체를 저장한다.
- 파일 1_2: 학습모델을 통해 추론(예측)을 개발하는 코드, 전처리 객체나 학습모델 객체를 불러와서 추론을 수행한다. 

- 파일 1_1(파일명): 1_1_local_platform_iris_classification_preprocess_train_tensorflow.ipynb
- 파일 1_2(파일명): 1_2_local_platform_iris_classification_inference_tensorflow.ipynb

### 전처리 객체 또는 학습모델 객체
- 전처리 객체나 학습모델 객쳬는 meta_data 폴더 아래에 저장한다.

### 데이터 세트(학습 데이터/테스트 데이터)
- 학습과 테스트에 사용되는 데이터를 나누어 관리한다.
- 학습 데이터: dataset 폴더 아래에 저장하거나 dataset.zip 파일 형태로 저장한다.
- 테스트 데이터: test_dataset 폴더 아래에 저장하거나 test_dataset.zip 파일 형태로 저장한다.

### 로컬 개발 워크플로우(workflow)  
- 로컬 개발 워크플로우를 다음의 4단계로 분리한다.

#### 파일 1_1
1. **데이터 세트 준비(Data Setup)**
- 로컬 저장소에서 전처리 및 학습에 필요한 학습 데이터 세트를 준비한다.

2. **데이터 전처리(Data Preprocessing)**
- 데이터를 모델 학습에 사용할 수 있도록 가공한다.
- 추론과정에서 필요한 경우, 데이터 전처리에 사용된 객체를 meta_data 폴더 아래에 저장한다.

3. **학습 모델 훈련(Train Model)**
- 데이터를 훈련에 사용할 수 있도록 가공한 뒤에 학습 모델을 구성한다. 
- 학습 모델을 준비된 데이터 세트로 훈련시킨다.
- 정확도(Accuracy)등 학습 모델의 성능을 검증한다.
- 학습 모델의 성능 검증 후, 학습 모델을 배포한다.
- 배포할 학습 모델을 meta_data 폴더 아래에 저장한다.

#### 파일 1_2
4. **추론(Inference)**
- 저장된 전처리 객체나 학습 모델 객체를 준비한다.
- 추론에 필요한 테스트 데이터 세트를 준비한다.
- 배포된 학습 모델을 통해 테스트 데이터에 대한 추론을 진행한다. 

# IRIS 분류( Classification)
- iris 꽃 데이터 분류

In [12]:
import tensorflow as tf
import logging
import os
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import pickle

## 1. **데이터 세트 준비(Data Setup)**

In [13]:
# 로컬에 저장된 iris 데이터 csv 파일을 읽어온다.
df=pd.read_csv('dataset/iris_classification_dataset.csv')

## **2. 데이터 전처리(Data Preprocessing)**

In [14]:
# 문제 데이터와 정답 데이터 나누기
X=df.iloc[:,:-1]
y=df.iloc[:,-1:]

In [15]:
# LabelEncoder를 이용해서 'Iris-setosa', 'Iris-versicolor', 'Iris-virginica'를 각각 0, 1, 2의 숫자형 레이블로 변환
label_encoder = LabelEncoder()
y=label_encoder.fit_transform(y)

C:\Users\USER\anaconda3\envs\nyh_tf\lib\site-packages\sklearn\preprocessing\_label.py:115: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [16]:
# 과적합 방지를 위한 데이터 셋 분리
X_train, X_test, Y_train, Y_test = train_test_split(X,y,test_size=0.2,random_state=0)

In [17]:
# 추론을 위한 데이터 저장.
with open(os.path.join('meta_data/label_encoder.p'), 'wb') as f:
    pickle.dump(label_encoder, f)    

## 3. 모델 설계 및 훈련

In [ ]:
    model =tf.keras.models.Sequential(    [
      tf.keras.layers.Dense(16,input_dim=4, activation='relu'),  
      tf.keras.layers.Dense(32, activation='relu'),  
      tf.keras.layers.Dense(10, activation='relu'),
      tf.keras.layers.Dense(3, activation='softmax')
    ])

    model.compile(optimizer='adam',
                loss='sparse_categorical_crossentropy',
                metrics=['accuracy'])

In [18]:
model.fit(X_train, Y_train, epochs=100)

Epoch 1/500

If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

4/4 [==============================] - 0s 500us/step - loss: 4.3909 - accuracy: 0.3333
Epoch 2/500
4/4 [==============================] - 0s 500us/step - loss: 3.9170 - accuracy: 0.3167
Epoch 3/500
4/4 [==============================] - 0s 500us/step - loss: 4.0604 - accuracy: 0.3250
Epoch 4/500
4/4 [==============================] - 0s 500us/step - loss: 3.8076 - accuracy: 0.3167
Epoch 5/500
4/4 [==============================] - 0s 500us/step - loss: 3.2173 - accuracy: 0.3250
Epoch 6/500
4

## 4. 학습 model 저장하기

In [19]:
# 학습 모델 저장.
model_path = os.path.join('./meta_data', 'iris_model.h5')
# 모델 저장
model.save(model_path)